In [2]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
from src import prepare

In [3]:
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
post, thread = prepare.load_dfs('train')

post.head()

,thread_num,user,text,quotes
0,45016,Mephansteras,Basically this is where we talk about what ga...,[]
1,45016,dakarian,The currently running or about to run games i...,[]
2,45016,webadict,And mine s started I ll try to limit m...,[]
3,45016,ExKirby,Mine needs players not,[]
4,45016,RedWarrior0,Mine can wait a bit BYORPE is a problem as it...,[]


In [3]:
thread.head()

,thread_num,thread_name,thread_label,thread_replies,thread_label_id
0,45016,Games Threshold Discussion and List [Vote for ...,other,5703,8
1,88720,New Player's Guide to the Subforum - New to Ma...,other,961,8
2,39338,Mafia: A Basic Tutorial,other,79,8
3,34959,Paranormal Mafia Game - Rules Discussion,other,1719,8
4,64229,Notable Games Archive,other,307,8


In [4]:
thread = thread.set_index('thread_num')

In [5]:
label_map = solution.prepare.load_label_map()

NameError: name 'solution' is not defined

# Features

In [ ]:
def count_prob(x):    
    has_quotes = 0
    for i in x:
        if i:
            has_quotes += 1
    return has_quotes / x.size
    
def get_mean_len_text(post):
    
    mean_len_text = post.groupby('thread_num',sort=False)['text'].apply(lambda x: np.mean(x.str.len())).reset_index(name='mean_len_text')
    mean_len_text = mean_len_text.set_index('thread_num')
    return mean_len_text

def get_quotes_probability(post):
    quotes_probability = post.groupby('thread_num', sort=False)['quotes'].apply(count_prob).reset_index(name='quotes_probability')
    quotes_probability = quotes_probability.set_index('thread_num')
    return quotes_probability

mean_len_text = get_mean_len_text(post)
quotes_probability = get_quotes_probability(post)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1, 1), min_df=3)

In [ ]:
word_vectors_raw = cv.fit_transform(thread['thread_name'])

In [ ]:
word_df = pd.DataFrame(word_vectors_raw.toarray(), columns=cv.get_feature_names(), index=thread.index)
word_df.head()

In [ ]:
X = pd.concat([
        mean_len_text['mean_len_text'],
        quotes_probability['quotes_probability'],
        (thread['thread_replies'] + 1).rename('posts'), 
        word_df,
    ], axis='columns')
X.head()

In [ ]:
y = thread['thread_label_id']

# Split


In [ ]:
from sklearn.model_selection import train_test_split

validation_pct = 0.25
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_pct, random_state=99)

# Fit


In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 120)
forest.fit(X_train, y_train)


In [ ]:
y_train_pred = pd.Series(
    forest.predict(X_train), 
    index=X_train.index, 
)
y_val_pred = pd.Series(
    forest.predict(X_val), 
    index=X_val.index, 
)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
def confusion_df(y_actual, y_pred):
    res = pd.DataFrame(
        confusion_matrix(y_actual, y_pred, labels=label_map.values),
        index=label_map.index.rename('predicted'),
        columns=label_map.index.rename('actual'),
    )
    return res

In [ ]:
confusion_df(y_val, y_val_pred).style.highlight_max()

In [ ]:
print("Test accuracy:", accuracy_score(y_train, y_train_pred))
print("Validation accuracy:", accuracy_score(y_val, y_val_pred))

In [ ]:
report = classification_report(y_val, y_val_pred, labels=label_map.values, target_names=label_map.index)
print(report)

# Predict


In [ ]:
post_test, thread_test = solution.prepare.load_dfs('test')
thread_test = thread_test.set_index('thread_num')

In [ ]:
word_vectors_raw_test = cv.transform(thread_test['thread_name'])
word_df_test = pd.DataFrame(word_vectors_raw_test.toarray(), columns=cv.get_feature_names(), index=thread_test.index)


In [ ]:
mean_len_text_test = get_mean_len_text(post_test)
quotes_probability_test = get_quotes_probability(post_test)

In [ ]:
X_test = pd.concat([
        mean_len_text_test['mean_len_text'],
        quotes_probability_test['quotes_probability'],
        (thread_test['thread_replies'] + 1).rename('posts'), 
        word_df_test,
    ], axis='columns')
X_test.head()

In [ ]:
forest.fit(X, y)

In [ ]:
y_test_pred = pd.Series(
    forest.predict(X_test), 
    index=X_test.index, 
)

In [ ]:
result = thread_test.copy()
result['thread_label_id'] = y_test_pred
result = result.reset_index()[['thread_num', 'thread_label_id']]

# Export

In [ ]:
out_dir = os.path.abspath('output')
os.makedirs(out_dir, exist_ok=True)
result.to_csv(
    os.path.join(out_dir, 'min_df&my_features.csv'),
    index=False, header=True, encoding='utf-8', 
)